1. Install
2. Download (part of?) water dataset.. let's try like this, if too complex then we can go back to storing it.
3. Run autotune. How to run hydra from python? See [docs](https://github.com/facebookresearch/hydra/blob/main/examples/advanced/ad_hoc_composition/hydra_compose_example.py)
4. compute errors
5. Run simple ASE MD for a few steps using the produced model.
6. Plot RDF!

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from tqdm.auto import tqdm

from franken.autotune import autotune
from franken.datasets.registry import DATASET_REGISTRY
from franken.trainers.rf_cuda_lowmem import RandomFeaturesTrainer
from franken.data.base import BaseAtomsDataset
from franken.backbones.utils import CacheDir
from franken.rf.model import FrankenPotential
import franken.metrics
from franken.data.base import Target
from franken.config import MaceBackboneConfig, MultiscaleGaussianRFConfig
from franken.config import AutotuneConfig, DatasetConfig, HPSearchConfig, SolverConfig

We use the H2O data obtained from DFT calculations (using RPBE+D3 theory), originally collected by [Montero de Hijes et al.](https://doi.org/10.1063/5.0197105).
To showcase the sample efficiency of our model, we finetune a the MACE-L0 foundation model (which in a zero-shot setting has pure accuracy on this data) with only 8 new samples.

### Configure

### Load the data

Here we load 8 random samples from the training set, and the validation set which contains 100 structures.

The dataset will be downloaded into franken's cache directory (`CacheDir.get()`).


> NOTE: The **cache directory** is used to store downloaded datasets and model backbones.
> it defaults to `$HOME/.franken` for the current user and it can be configured by setting the `$FRANKEN_CACHE_DIR`
> environment variable.

In [ ]:
# We need to configure the GNN backbone we wish to use
# since the data format depends on which backbone will be loaded.
# We will use the MACE-L0 backbone, with features extracted at the 2nd layer
gnn_config = MaceBackboneConfig(
    path_or_id="MACE-L0",
    interaction_block=2,
)

In [ ]:
train_dset_8 = BaseAtomsDataset.from_path(
    data_path=DATASET_REGISTRY.get_path("water", "train", base_path=CacheDir.get()),
    split="train",
    num_random_subsamples=8,
    subsample_rng=42,
    gnn_config=gnn_config,
)
train_dl_8 = train_dset_8.get_dataloader(distributed=False)

cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.


ASE -> MACE (train):   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
val_dset = BaseAtomsDataset.from_path(
    data_path=DATASET_REGISTRY.get_path("water", "val", base_path=CacheDir.get()),
    split="val",
    gnn_config=gnn_config,
)
val_dl = val_dset.get_dataloader(distributed=False)

ASE -> MACE (val):   0%|          | 0/189 [00:00<?, ?it/s]

### Training a single model

We start in the most explicit setting where all the hyperparameters are set before-hand. Below we describe them one by one in detail.

The first two parameters describe the **backbone model**: MACE-L0, with features extracted at the 2nd layer

In [5]:
gnn_backbone_id = "MACE-L0"
interaction_block = 2

Next we **set the kernel**, and its hyperparameters.
Here we use the multiscale Gaussian kernel, which can fit a range of length-scales $\sigma$ with a single model.
This has the big advantage of requiring little tuning: a range of sensible values of $\sigma$ can be set through the three parameters `"length_scale_low", "length_scale_high", "length_scale_num"`. This is much easier than precisely setting the optimal length0scale, and retains most of the accuracy.

The other parameter is the number of random features, which can generally be set based on how much computing time and memory is available: more random features increase accuracy monotonically, although diminishing returns kick in after $\approx 16000$ random features.

In [ ]:
rf_config = MultiscaleGaussianRFConfig(
    num_random_features=512,
    length_scale_low=4.0,
    length_scale_high=24.0,
    length_scale_num=4,
    rng_seed=42,  # for reproducibility
)

The **linear system hyperparameters** are:
 - The L2 regularization weight (`"L2_penalty"`), which should be a small number and can be increased if numerical issues arise.
 - The weight of the forces compared to energies in the overall loss. If `"loss_lerp_weight"` is closer to 1, the forces have more weight -- which is typically the desired configuration -- while when it is set closer to 0, the energies have more weight.

 It is generally very easy and fast to do a hyperparameter search on these parameters as we will see later in this notebook.
 For the sake of simplicity here they are defined explicitly.

In [ ]:
solver_params = {
    "l2_penalty": [1e-5],
    "loss_lerp_weight": [0.8],
}

Instantiate classes for the model and for the training algorithm.

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = FrankenPotential(
    gnn_config=gnn_config,
    rf_config=rf_config,
    num_species=2,        # H and O
    jac_chunk_size=12,    # chosen to fit in the T4 GPU on colab. You can set it to "auto" to adapt to available GPU memory.
)
trainer = RandomFeaturesTrainer(
    train_dataloader=train_dl_8,
    save_every_model=False,
    device=device,
    save_fmaps=False,
)

Call the `fit` method to **train the model**. A separate model for all parameter possibilities in the `solver_params` dictionary is trained, but note that in this case a single possibility has been specified.

In [20]:
logs, weights = trainer.fit(model, solver_params)

Patching e3n '_spherical_harmonics' function failed unexpectedly. This may or may not be a problem.


Computing dataset statistics:   0%|          | 0/8 [00:00<?, ?it/s]

    covs+coeffs      | 0.3 cfgs/s |  88% | 1 x NVIDIA RTX 6000 Ada Generation

`leading_eig` normalization has high memory usage. If you encounter OOM errors try to disable it.


   least-squares     | 910.4 models/s | 1 x NVIDIA RTX 6000 Ada Generationion

Run the model to get **predictions** on the whole validation set. We record simple metrics such as energy and force MAE.

> NOTE: The **forces_mode** parameter can be set to either "torch.func" or "torch.autograd". The two have different performance characteristics: "torch.func" is more suitable when there are many different models being trained (i.e. many different solver hyperparameters) as it can effectively run these in a batch way. When only a few models are present, "torch.autograd" can be much faster. In our testing the cutoff between the two is approximately at 100 models, but this may vary greatly depending on hardware characteristics.

In [ ]:
energy_mae = franken.metrics.init_metric("energy_MAE", device=device)
forces_mae = franken.metrics.init_metric("forces_MAE", device=device)
for atom_data, targets in tqdm(val_dl, desc="Predicting energies and forces"):
    # Move the data to the GPU
    atom_data = atom_data.to(device=device)
    targets = targets.to(device=device)
    pred_e, pred_f = model.energy_and_forces(
        atom_data,
        weights=weights,
        forces_mode="torch.autograd",
        add_energy_shift=True
    )
    predictions = Target(pred_e, pred_f)
    energy_mae.update(predictions, targets)
    forces_mae.update(predictions, targets)
energy_mae = energy_mae.compute()
forces_mae = forces_mae.compute()

Predicting energies and forces:   0%|          | 0/189 [00:00<?, ?it/s]

In [15]:
print(f"Energy error = {energy_mae} eV/atom")
print(f"Forces error = {forces_mae} eV/atom")

Energy error = tensor([1.3339, 1.6966, 9.4363, 0.6200, 0.6260, 1.1368, 0.4391, 0.4357, 0.4207,
        0.3163, 0.3162, 0.3226, 0.3731, 0.3927, 0.4086, 0.3977, 0.4172, 0.4297,
        0.4022, 0.4164, 0.4399], device='cuda:0') eV/atom
Forces error = tensor([35.2299, 34.8653, 34.5746, 29.6398, 29.4364, 29.2765, 26.7001, 26.6049,
        26.5321, 25.6087, 25.5922, 25.5748, 25.5772, 25.5806, 25.5855, 25.6596,
        25.6493, 25.6663, 25.7061, 25.6690, 25.6730], device='cuda:0') eV/atom


### Hyperparameter tuning

Now we will use the automatic hyperparameter tuning interface to franken which can find
the best solver hyperparameters, as well as the best kernel parameters. The `autotune` interface
can be used from scripts like here or from the command line (through the `franken.autotune` command), 
and also takes care of saving the trained model to disk.

Equivalent of running:
```bash
franken.autotune preset=gaussian-MACE-L0 \
    dataset.dataset_name='water' \
    dataset.train_subsample.num=8 \
    dataset.train_subsample.rng_seed=42 \
    hyperparameters.random_features.length_scale.start=1 \
    hyperparameters.random_features.length_scale.stop=30 \
    hyperparameters.random_features.length_scale.num=10
```

In [ ]:
dataset_cfg = DatasetConfig(name="water", max_train_samples=8)
solver_cfg = SolverConfig(
    l2_penalty=HPSearchConfig(start=-11, stop=-5, num=10, scale='log'),  # equivalent of numpy.logspace
    force_weight=HPSearchConfig(start=0.01, stop=0.99, num=10, scale='linear'),  # equivalent of numpy.linspace
)
autotune_cfg = AutotuneConfig(
    dataset=dataset_cfg,
    solver=solver_cfg,
    backbone=gnn_config,
    rfs=rf_config,
    seed=42,
    run_dir="."
)

autotune(autotune_cfg)

2025-03-27 10:33:40.078 DEBUG (rank 0): console_logging_level: DEBUG
distributed: True
seed: 1337
dataset:
    dataset_name: water
    test_path: null
    train_path: null
    train_subsample:
      num: 8
      rng_seed: 42
    val_path: null
franken:
    atomic_energies: null
    gnn_backbone_id: MACE-L0
    interaction_block: 2
    jac_chunk_size: auto
    kernel_type: multiscale-gaussian
    scale_by_Z: true
hyperparameters:
    random_features:
      length_scale:
        _target_: numpy.linspace
        num: 10
        start: 1
        stop: 10
      length_scale_high: 30
      length_scale_low: 1
      length_scale_num: 5
      num_random_features: 1024
    solver:
      L2_penalty:
        _target_: numpy.logspace
        num: 6
        start: -11
        stop: -6
      loss_lerp_weight:
        _target_: numpy.linspace
        num: 3
        start: 0.01
        stop: 0.99
paths:
    root_dir: .
    run_dir: ./experiments/water/sample_complexity/MACE-L0-2/1024_rfs/8_subsamples/

ASE -> MACE (train):   0%|          | 0/8 [00:00<?, ?it/s]

ASE -> MACE (val):   0%|          | 0/189 [00:00<?, ?it/s]

2025-03-27 10:33:43.375 DEBUG (rank 0): Autotune iteration with RF parameters {'length_scale': 1.0, 'length_scale_high': 30, 'length_scale_low': 1, 'length_scale_num': 5, 'num_random_features': 1024}
2025-03-27 10:33:43.445 WARNING (rank 0): Tried to initialize MultiScaleOrthogonalRFF with invalid parameters: {'length_scale': 1.0}


Computing dataset statistics:   0%|          | 0/8 [00:00<?, ?it/s]

After running autotune the model has been saved to disk, along with useful statistics about the autotune process.
We will plot these statistics below.

### Molecular dynamics

The saved potentials model can now be used for a simple molecular dynamics experiment. 

In [ ]:
import ase
import ase.md
import ase.io
import ase.units

import logging
from pathlib import Path

from franken.mdgen.mdgen_utils import FrankenMDLogger
from franken.utils.misc import setup_logger
from franken.calculators.ase_calc import FrankenCalculator

In [ ]:
setup_logger(logging.INFO)
md_length_ns = 0.5
timestep_fs = 0.5
temperature_K = 325
friction = 0.01

In [ ]:
data_path = DATASET_REGISTRY.get_path("water", "train", CacheDir.get())
initial_configuration = ase.io.read(data_path, index=567)
# Trajectory will contain the output data
output_path = Path(".")
trajectory_path = output_path / f"md_output.traj"
trajectory = ase.io.Trajectory(trajectory_path, "w", initial_configuration)

In [ ]:
model_path = Path(".")
calc = FrankenCalculator(model_path, device=device)
integrator = ase.md.Langevin(
    temperature_K=temperature_K,
    friction=friction / ase.units.fs,
    atoms=initial_configuration,
    timestep=timestep_fs * ase.units.fs
)
md_logger = FrankenMDLogger(
    dynamics=integrator,
    atoms=initial_configuration,
    stress=False,
    peratom=False,
)
# nan checker needs to go first to be effective.
integrator.attach(md_logger, interval=100)
integrator.attach(trajectory.write, interval=100)

In [ ]:
integrator.run(md_length_ns * 1e6 / timestep_fs)